<a href="https://colab.research.google.com/github/hrithinnnn/survival-in-titanic/blob/main/TitanicSurvival_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
---
The goal of this tutorial is to familiarize yourself with basics of data set analysis, feature engineering and model training/ptrdiction. We will look at a simple Titanic survival prediction dataset.

# Task description
---
The task at hand is a simple classification as to whether a passenger on The Titanic survived or not based on the various features available in the dataset.

# Dataset
---
We will use the dataset from the Kaggle task [here](https://www.kaggle.com/c/titanic/data?select=train.csv). This is a very basic dataset, used only for illustrative purposes.

Lets go ahead and initialize the data.

## Initialize data

In [ ]:
# Create data directory and download train and test split csv's.
%mkdir data
%cd data
!wget -O train.csv https://drive.google.com/uc?id=1bbUXGCwVKhO0zak-FCkVbAOC6Qwk-UCA&export=download
!wget -O test.csv https://drive.google.com/uc?id=1ksmfNosC2Q14MSXfcKj8N_2oOWg3hPVL&export=download
%cd ..

In [ ]:
# All installs here
%pip install pandas
%pip install seaborn
%pip install matplotlib
%pip install numpy
%pip install scikit-learn

In [ ]:
# All imports here
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
import numpy as np
from random import randint

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from IPython.display import display

### Load data

In [ ]:
train_df = pd.read_csv("/content/data/train.csv")
test_df = pd.read_csv("/content/data/test.csv")

### Check meta data and columns available

In [ ]:
train_df.info(verbose=True, show_counts=False)
print()
test_df.info(verbose=True, show_counts=False)

### Sample data view

In [ ]:
train_df.head()

## Exploratory data analysis (EDA)

Given any dataset, EDA is the process to systematically analyze the data for any outliers and/or inconsistencies. This is also an opportunity to weed out any unuseful data which otherwise might hinder the learning process or contribute no additional supervision signals to the training.

### High level statistics
As we have loaded our train and test splits into Pandas, we can easily view high level statistics of our data as shown below.

#### Numerical features

In [ ]:
train_df.describe()

#### Categorical features

In [ ]:
train_df.describe(include=["O"])

### Correlation of categorical features with the target variable
Next we visualize the correlation of categorical features available in the dataset with the target variable **`Survived`**. This would give us a clear picture on the most useful features to select from the given set of features.

Also the correlation patterns will help us weed out any dependent feature patterns, i.e say if one of the feature is just a linear extrapolation of the other, then the training will be heavily biased and overfit that pattern. So we need to identify such features and isolate them.

#### Passenger class vs Survived

In [ ]:
display(
    train_df[["Pclass", "Survived"]]
    .groupby("Pclass", as_index=False)
    .mean()
    .sort_values(by="Survived", ascending=False)
)

From the above we can see that passengers travelling in **1**<sup>st</sup> class have had a higher probability of survival than those travelling in **3**<sup>rd</sup> with those in **2**<sup>nd</sup> class having an equal probability of surviving or otherwise.  
<br>

---
> 💡 **Note:** This is the probability of survival not the absolute count of people that survived the disaster.

#### Sex vs Survived

In [ ]:
display(
    train_df[["Sex", "Survived"]]
    .groupby("Sex", as_index=False)
    .mean()
    .sort_values(by="Survived", ascending=False)
)

Female passengers had a higher survival rate. A speculative reason could be that they were the ones evacuated first.

#### **Quiz 🎯:** Sibling/Spouse count vs Survived

In [ ]:
# Write a script here to obtain correlation between Sibling/Spouse count and the survival rate.
# Based on this correlation arrive at a glaring fact that the data shows you.
# column name: SibSp
# Write your script here
display(
    train_df[["SibSp", "Survived"]]
    .groupby("SibSp", as_index=False)
    .mean()
    .sort_values(by="Survived", ascending=False)
)

Fill the facts inferred from your graph here

### Correlation of numerical features with the target variable
Next we move on to the numerical features and analyze its distribution with respect to the target variable i.e the survival rate. This is different from categorical features as its not easy to visualize them as tables. Instead we try and visualize them as plots.

#### Passenger class vs Parent/Child count vs Sex vs Survived

In [ ]:
g = sns.FacetGrid(train_df, col="Survived")
g.map(
    sns.pointplot,
    "Pclass",
    "Parch",
    "Sex",
    pallete="pastel",
    order=[1, 2, 3],
    hue_order=["male", "female"],
    dodge=True,
)
g.add_legend()

We can infer the below facts from the plots:
1. Majority of the **1**<sup>st</sup> class passengers were travelling with their family.
2. Most of the **1**<sup>st</sup> class passengers who **survived were travelling alone**.
3. The survival rate of people travelling by **2**<sup>nd</sup> class does not have much correlation with the number of people travelling with them.
4. Most of the **female** passengers were travelling alone but that does not seem to contribute much for or against their survival rate.


> 🎯 **Quiz:** These are just some of the interpretations that I could infer from the above graph.  
> **Can you try and infer more conclusions ?**



#### Age vs Survived

In [ ]:
g = sns.FacetGrid(train_df, col="Survived")
g.map(plt.hist, "Age", bins=20)

Below are some of the facts inferred from the graph:

1. Infants and children upto **18** years of age seem to have had the highest **survival rate**.
2. The **highest number of casualities** were in middle age group of people.
3. Compared to middle age people survival rate, the survival rate of elder people have had been higher.



#### **Quiz 🎯:** Passenger class vs Sibling/Spouse count vs Sex vs Survived

In [ ]:
# Write a script to plot a graph between Passenger class, Sibling count and
# Sex for each Survival class and infer facts out of it.

g = sns.FacetGrid(train_df, col="Survived")
g.map(
    sns.pointplot,
    "Pclass",
    "SibSp",
    "Sex",
    pallete="pastel",
    order=[1, 2, 3],
    hue_order=["male", "female"],
    dodge=True,
)
g.add_legend()

Fill the facts inferred from your graph here

#### Age vs Travel class vs Sex correlation

In [ ]:
g = sns.FacetGrid(train_df, row="Pclass", col="Sex", aspect=1.6)
g.map(plt.hist, "Age", bins=20)
g.add_legend()

From the above we can see that there is quite some correlation between the age bucket counts vs the combination of travel class and sex.  

This is something we will be using down the line to categorize one of the features.

### Analyzing seemingly unusable features
Now we look into the different non-numerical/non-categorical features that are available in the given data. These include the following fields:
1. Name
2. Cabin
3. Ticket
4. Fare
  

Among the above, **cabin and ticket columns logically do not add any specific value** to our task at hand. So we decide to safely drop those fields from our feature space.  


With respect to **Fare, we can convert it to a categorical feature** (by defining buckets of fare and identifying them with specific constants) which will be a supplementary signal to the class in which a passenger travels. We will look into in next section.  


The Name field also does not add much value but on looking into the values for this field we can see that **most of them have salutations** embedded in it which can act as a supplementary signal to the existing **Sex** field.  


We will **extract salutations** from the name fields and then **define a derived categorical feature from their values**.

#### Dropping fields that are really unusable

In [ ]:
# Dropping the feature fields that are not required
train_df = train_df.drop(["PassengerId", "Ticket", "Cabin"], axis=1)
display(train_df.head())
print()
test_df = test_df.drop(["PassengerId", "Ticket", "Cabin"], axis=1)
display(test_df.head())

#### Converting **Name** to a valid categorical feature i.e **Title**

In [ ]:
train_df["Title"] = train_df.Name.str.extract(" ([A-Za-z]+)\.", expand=False)
test_df["Title"] = test_df.Name.str.extract(" ([A-Za-z]+)\.", expand=False)
display(train_df.head())

print("\n\nDistribution of titles among different people:")
# Check the distribution of titles among different people
display(pd.crosstab(train_df["Title"], train_df["Sex"]))

We can clearly see that most of the titles fall into one or two values and the rest can be categorized into a common **Rare** bucket.  

Also few titles like **Mlle**, **Miss** etc can be safely replaced with more generic titles.

In [ ]:
# Grouping rare titles and converting few to more common ones
for dataset in [train_df, test_df]:
    dataset["Title"] = dataset["Title"].replace(
        [
            "Capt",
            "Col",
            "Countess",
            "Don",
            "Dr",
            "Jonkheer",
            "Lady",
            "Major",
            "Rev",
            "Sir",
        ],
        "Rare",
    )
    dataset["Title"] = dataset["Title"].replace(["Mlle", "Ms"], "Miss")
    dataset["Title"] = dataset["Title"].replace("Mme", "Mrs")

display(train_df[["Title", "Survived"]].groupby(["Title"], as_index=False).count())

Convert the final **Titles** to integer values

In [ ]:
TITLE_MAP = {
    title: idx for idx, title in enumerate(train_df["Title"].unique().tolist())
}

for dataset in [train_df, test_df]:
    dataset["Title"] = dataset["Title"].map(TITLE_MAP).fillna(0).astype(int)

train_df = train_df.drop("Name", axis=1)
test_df = test_df.drop("Name", axis=1)

display(train_df.head())
print()
display(test_df.head())

#### Convert string categorical features (namely **Sex** and **Embarked**) to integer features.

In [ ]:
SEX_MAP = {"female": 1, "male": 0}
EMBARK_MAP = {"S": 0, "C": 1, "Q": 2}

for dataset in [train_df, test_df]:
    dataset["Sex"] = dataset["Sex"].map(SEX_MAP).astype(int)

    if dataset["Embarked"].hasnans:
        """
        if there are NaN values then fill it with the most recurring
        value for Embarked
        """
        dataset["Embarked"] = dataset["Embarked"].fillna(
            train_df["Embarked"].value_counts().idxmax()
        )
    dataset["Embarked"] = dataset["Embarked"].map(EMBARK_MAP).astype(int)

train_df.head()

#### Complete continous numeric valued features
Some numeric features have NaN values. We need to remove all NaN values and populate with possible best guess values for them.  

##### **Quiz 🎯:** We first need to find the columns with NaN values.

In [ ]:
# Write a short script to print all columns in train and
# test data that have atleast one NaN value
#for dataset in [train_df, test_df]:
dataset.isna().any()[lambda x: x]


##### Handling **NaN**'s in **Age** field
From the given feature space, the best guess for the age of a person can be made from the combination of the travelling class and the sex of the person travelling.  

This is corroborated from the findings from our EDA above in the section **Age vs Travel class vs Sex correlation**.

So we compute the median age for each of the four combinations of travel class and sex and assign it to the NaN's from the same feature values.

In [ ]:
for dataset in [train_df, test_df]:
    for i in range(0, 2):
        for j in range(0, 3):
            age_guess = (
                dataset[(dataset["Sex"] == i) & (dataset["Pclass"] == j + 1)]["Age"]
                .dropna()
                .median()
            )
            dataset.loc[
                (dataset["Age"].isnull())
                & (dataset["Sex"] == i)
                & (dataset["Pclass"] == j + 1),
                "Age",
            ] = (
                int(age_guess * 2 + 0.5) * 0.5
            )

    dataset["Age"] = dataset["Age"].astype(int)

display(train_df["Age"].hasnans)
display(test_df["Age"].hasnans)

##### Handling **NaN**'s in **Fare** field in the test set
As there are very less instances of NaN's, we simply take the median value and replace the NaN's with it.

In [ ]:
test_df["Fare"].fillna(test_df["Fare"].dropna().median(), inplace=True)
display(test_df["Fare"].hasnans)

#### Convert continuous numeric features to categories
It is always better to bin continous float valued features to specific number of bins. This allows the model to be more robust to any variations in the feature and any anomalies in the absolute value of a feature.  

We will use a simple trick where we compute the quartile ranges of the field and assign an integer to each quartile range.

##### Handling the continuous feature **Fare** to categorical

In [ ]:
# We intend to create 4 quartile bands for the continuous
# Use a Pandas function to assign the inter quartile range
# values of Fare as a new column FareBand

train_df["FareBand"] = pd.qcut(train_df["Fare"], 4)
train_df[["FareBand", "Survived"]].groupby(
    ["FareBand"], as_index=False
).mean().sort_values(by="FareBand", ascending=True)

In [ ]:
def convert_fare_to_band(fare: float):
    if fare <= 7.91:
        return 0
    elif fare > 7.91 and fare <= 14.454:
        return 1
    elif fare > 14.454 and fare <= 31:
        return 2
    elif fare > 31:
        return 3


for dataset in [train_df, test_df]:
    dataset["Fare"] = dataset["Fare"].map(convert_fare_to_band)

# Drop the temporary column created before
train_df = train_df.drop(["FareBand"], axis=1)
train_df.head()

##### **Quiz 🎯:** Handling the **Age** feature to convert it into bands
Similar to the **Fare** feature, we need to create inter-quartile ranges and map all the age values that lie within each of the quartile with a specific integer value.  

To do this implement the below as per guidance

In [ ]:
# Create a temporary feature column "AgeBand" in the training
# data frame for 5 quartiles.
train_df["AgeBand"] = pd.qcut(train_df["Age"], 5)
#train_df["AgeBand"] = ???
train_df[["AgeBand", "Survived"]].groupby(["AgeBand"], as_index=False).mean().sort_values(by="AgeBand", ascending=True)

In [ ]:
# Create a python function that maps each range of AgeBand to a unique
# incrementing class value
def convert_age_to_band(age: float):
    # dummy value
    ret_val: int = -999

    if age <= 20:
        return 0.0
    elif age > 21 and age <= 40:
        return 1.0
    elif age > 41 and age <= 60:
        return 2.0
    elif age > 61 and age <= 80:
        return 3.0
    elif age>81:
        return 4.0;

    return ret_val


for dataset in [train_df, test_df]:
    dataset["Age"] = dataset["Age"].map(convert_age_to_band)

# Uncomment below line after implementing the previous two cells
train_df = train_df.drop(["AgeBand"], axis=1)

## Final data
The final training and test data is displayed below. We have converted the features to their categorical values.  

We will now proceed with model training with this training and test split data.

In [ ]:
display(train_df.head())
display(test_df.head())

# Model families and Training
---

We will now create different model families and test our data on them and compare their performance. 

The task is to train a model with the available training data i.e supervised learning. We explore a combination of regression and classifier models as our problem can be modelled as either a classification or a regression task. 

To keep things simple we will explore the below models:
1. Naive Bayes classifier
2. Logistic regression
3. Decision trees
4. Random Forest classifier

## Data preparation

We now prepare the data into input and output variables in order to be able to give the models its corresponding inputs.

In [ ]:
X_train = train_df.drop(["Survived"], axis=1)
Y_train = train_df["Survived"]

X_test = test_df

## Naive Bayes classifier

A Gaussian Naive Bayes classifier are a simple family of probabilistic models that work on the core principle of Baye's theorem.  

In [ ]:
gaussian_nb = GaussianNB()
gaussian_nb.fit(X_train, Y_train)
Y_pred = gaussian_nb.predict(X_test)
print([Y_pred[idx] for idx in range(0, 5)])

acc_gnb = round(gaussian_nb.score(X_train, Y_train) * 100.0, 2)
acc_gnb

## Logistic regression
Logistic regression models the correlation between categorical input feature variables to that of one or more independent output variables by estimating a logistic function.  

This means it tries to fit a generic S-shaped sigmoid curve not necessarily having a mean of 0 and standard deviation of 1. This generic logistic curve tries to capture the inherent function of the independent variables that we are trying to model.  

In [ ]:
logistic_reg = LogisticRegression()
logistic_reg.fit(X_train, Y_train)
Y_pred = logistic_reg.predict(X_test)
print([Y_pred[idx] for idx in range(0, 5)])

acc_log_reg = round(logistic_reg.score(X_train, Y_train) * 100.0, 2)
acc_log_reg

## **Quiz 🎯:** Decision trees
This family of models basically map the problem as a tree where the branches constitute the input features and the output features or classes are part of the leaf node.  

They can either be classification trees or regression trees depending on the output values. In this case we will be using a classification tree.  


In [ ]:
decision_tree = DecisionTreeClassifier()

decision_tree.fit(X_train, Y_train)

# Run predictions on X_test using the model
Y_pred = decision_tree.predict(X_test)

# Once implemented uncomment all the statements below
print([Y_pred[idx] for idx in range(0, 5)])

acc_dtree =  round(decision_tree.score(X_train, Y_train) * 100.0, 2)
acc_dtree

## Random forest classifier
A random forest classifier models the decision using an ensemble of decision trees. The final prediction class is obtained by taking the mode (max frequency) of the classes predicted by all the decision trees.  

In [ ]:
random_fc = RandomForestClassifier(n_estimators=200)
random_fc.fit(X_train, Y_train)
Y_pred = random_fc.predict(X_test)
print([Y_pred[idx] for idx in range(0, 5)])

acc_rfc = round(random_fc.score(X_train, Y_train) * 100.0, 2)
acc_rfc

## Model performance summary
We now come to the end of this EDA and Model exploration and tabulate the performance of each type of model on the training set.  

In [ ]:
models = pd.DataFrame(
    {
        "Model": [
            "Naive Bayes",
            "Logistic Regression",
            "Random Forest",
            "Decision Trees",
        ],
        "Accuracy": [acc_gnb, acc_log_reg, acc_rfc, acc_dtree],
    }
)
models = models.sort_values(by="Accuracy", ascending=False).reset_index(drop=True)
display(models)